In [96]:
import awkward as ak
import numpy as np
import uproot as up
import hist
import correctionlib
from coffea.lookup_tools import extractor

In [102]:
ext = extractor()
# several histograms can be imported at once using wildcards (*)
ext.add_weight_sets(["PtEta_2dWeight PtEta_2dWeight 2DWeight_pt_eta.root"])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)
print("PtEta_2dWeight:", evaluator['PtEta_2dWeight'])
print("type of PtEta_2dWeight:", type(evaluator['PtEta_2dWeight']))

available evaluator keys:
	 PtEta_2dWeight
PtEta_2dWeight: 2 dimensional histogram with axes:
	1: [ 15.  25.  35.  45.  55.  65.  75.  85.  95. 105. 115. 125. 135. 145.
 155. 165. 175. 185. 195. 205. 215. 225. 235. 245. 255. 265. 275. 285.
 295. 305. 315. 325. 335. 345. 355. 365. 375. 385. 395. 405. 415. 425.
 435. 445. 455. 465. 475. 485. 495. 505. 515. 525. 535. 545. 555. 565.
 575. 585. 595. 605. 615.]
	2: [0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.    1.125 1.25  1.375
 1.5   1.625 1.75  1.875 2.    2.125 2.25  2.375 2.5  ]

type of PtEta_2dWeight: <class 'coffea.lookup_tools.dense_lookup.dense_lookup'>


In [86]:
events = up.open("out_GJet_PT-15to6000_TuneCP5-Flat_13p6TeV_20to29.root:tree")
print(events)
tot_events = events.arrays(["photon_sieip",
                            "photon_sieie",
                            "photon_r9",
                            "photon_etaWidth",
                            "photon_phiWidth",
                            "photon_esEnergyOverRawE",
                            "photon_esEffSigmaRR",
                            "photon_pfPhoIso03",
                            "photon_hoe",
                            "photon_ecalPFClusterIso",
                            "photon_hcalPFClusterIso",
                            "photon_energyRaw",   
                            "photon_pfChargedIsoPFPV",
                            "photon_pfChargedIsoWorstVtx",
                            "photon_s4",
                            "photon_sipip",
                            "photon_pt",
                            "photon_eta",
                            "photon_isScEtaEB",
                            "photon_isScEtaEE",
                            "isPhoton"])
print(type(tot_events))
#tot_df = ak.to_pandas(tot_events)
tot_sig = tot_events[tot_events.isPhoton==1]
tot_sig = tot_sig[ak.num(tot_sig.isPhoton)==1]
#eta_cut_sig = np.stack((tot_sig.photon_isScEtaEB, tot_sig.photon_isScEtaEE))
#tot_sig = tot_sig[ak.any(eta_cut_sig, axis=1)]
#tot_sig = tot_sig[ak.num(tot_sig.isPhoton)==1]

<TTree 'tree' (28 branches) at 0x7f5b0e5cd7b0>
<class 'awkward.highlevel.Array'>


In [87]:
tot_sig.photon_sieie

<Array [[0.00725], [0.00974, ... [0.00928]] type='5299160 * var * float32'>

In [88]:
# Load the correctionlib JSON file #
evaluator = correctionlib.CorrectionSet.from_file("../EGmSFs/SS.json")
print(list(evaluator.keys()))
# Obtain the smearing values
evaluator_smearing = evaluator["Prompt2022FG_SmearingJSON"]
rho = evaluator_smearing.evaluate("rho", tot_sig.photon_eta, tot_sig.photon_r9)
rho

['Prompt2022FG_ScaleJSON', 'Prompt2022FG_SmearingJSON']


array([[0.01588],
       [0.02175],
       [0.01588],
       ...,
       [0.01081],
       [0.01588],
       [0.01588]])

In [104]:
pt_eta_2DWeight = evaluator["PtEta_2dWeight"](tot_sig.photon_pt, tot_sig.photon_eta)
file = up.recreate('GJet_PT-15to6000_TuneCP5-Flat_13p6TeV_20to29_sig.root')
file["treeS"] = {"photon_sieip": tot_sig.photon_sieip[:,0],
                 "photon_sieie": tot_sig.photon_sieie[:,0],
                 "photon_r9": tot_sig.photon_r9[:,0],
                 "photon_etaWidth": tot_sig.photon_etaWidth[:,0],
                 "photon_phiWidth": tot_sig.photon_phiWidth[:,0],
                 "photon_esEnergyOverRawE": tot_sig.photon_esEnergyOverRawE[:,0],
                 "photon_esEffSigmaRR": tot_sig.photon_esEffSigmaRR[:,0],
                 "photon_pfPhoIso03": tot_sig.photon_pfPhoIso03[:,0],
                 "photon_hoe": tot_sig.photon_hoe[:,0],
                 "photon_ecalPFClusterIso": tot_sig.photon_ecalPFClusterIso[:,0],
                 "photon_hcalPFClusterIso": tot_sig.photon_hcalPFClusterIso[:,0],
                 "photon_energyRaw":   tot_sig.photon_energyRaw[:,0],
                 "photon_pfChargedIsoPFPV":tot_sig.photon_pfChargedIsoPFPV[:,0],
                 "photon_pfChargedIsoWorstVtx": tot_sig.photon_pfChargedIsoWorstVtx[:,0],
                 "photon_s4": tot_sig.photon_s4[:,0],
                 "photon_sipip": tot_sig.photon_sipip[:,0],
                 "photon_rho": rho[:,0],
                 "photon_pt": tot_sig.photon_pt[:,0],
                 "photon_eta": tot_sig.photon_eta[:,0],
                 "photon_isScEtaEB": tot_sig.photon_isScEtaEB[:,0],
                 "photon_isScEtaEE": tot_sig.photon_isScEtaEE[:,0],
                 "pt_eta_2DWeight": pt_eta_2DWeight[:,0],
                 "isPhoton": tot_sig.isPhoton[:,0]
                }
pho_pt = tot_sig.photon_pt[:,0]
pho_eta = tot_sig.photon_eta[:,0]
pho_isScEtaEB = tot_sig.photon_isScEtaEB[:,0]
pho_isScEtaEE = tot_sig.photon_isScEtaEE[:,0]

h_photon_pt_eb = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_eb").Weight()
h_photon_eta_eb = hist.Hist.new.Reg(10, 0, 1.44, name="photon_eta_eb").Weight()
h_photon_pt_ee = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_ee").Weight()
h_photon_eta_ee = hist.Hist.new.Reg(10, 1.566, 2.5, name="photon_eta_ee").Weight()
h_pt_eta_eb = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 0, 1.44))
h_pt_eta_ee = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 1.566, 2.5))
h_pt_eta = hist.Hist(hist.axis.Regular(60, 15, 615),hist.axis.Regular(20, 0, 2.5))

h_photon_pt_eb.fill(pho_pt[pho_isScEtaEB==1])
h_photon_pt_ee.fill(pho_pt[pho_isScEtaEE==1])
h_photon_eta_eb.fill(abs(pho_eta[pho_isScEtaEB==1]))
h_photon_eta_ee.fill(abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta_eb.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta_ee.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))

file["Photon_pt_EB"] = h_photon_pt_eb
file["Photon_pt_EE"] = h_photon_pt_ee
file["Photon_eta_EB"] = h_photon_eta_eb
file["Photon_eta_EE"] = h_photon_eta_ee
file["Photon_pt_eta_EB"] = h_pt_eta_eb
file["Photon_pt_eta_EE"] = h_pt_eta_ee
file["Photon_pt_eta"] = h_pt_eta

In [90]:
###For backgrounds
tot_bkg = tot_events[tot_events.isPhoton==0]
tot_bkg1 = tot_bkg[ak.num(tot_bkg.isPhoton)>0]
tot_bkg2 = tot_bkg[ak.num(tot_bkg.isPhoton)>1]
tot_bkg3 = tot_bkg[ak.num(tot_bkg.isPhoton)>2]

In [91]:
print(tot_bkg1.photon_sieie)
print(tot_bkg2.photon_sieie)
print(tot_bkg3.photon_sieie)

[[0.0199], [0.0139], [0.03], [0.0282, ... [0.0188], [0.00144], [0.0176, 0.0102]]
[[0.0282, 0.0186], [0.0291, 0.0101], ... [0.0112, 0.0142], [0.0176, 0.0102]]
[[0.0293, 0.00839, 0.0174], [0.0283, 0.0145, ... 0.00876], [0.00882, 0.0343, 0.0406]]


In [92]:
# Load the correctionlib JSON file #
evaluator = correctionlib.CorrectionSet.from_file("../EGmSFs/SS.json")
print(list(evaluator.keys()))
# Obtain the smearing values
evaluator_smearing = evaluator["Prompt2022FG_SmearingJSON"]
rho1 = evaluator_smearing.evaluate("rho", tot_bkg1.photon_eta[:,0], tot_bkg1.photon_r9[:,0])
rho2 = evaluator_smearing.evaluate("rho", tot_bkg2.photon_eta[:,1], tot_bkg2.photon_r9[:,1])
rho3 = evaluator_smearing.evaluate("rho", tot_bkg3.photon_eta[:,2], tot_bkg3.photon_r9[:,2])
print(rho1)

['Prompt2022FG_ScaleJSON', 'Prompt2022FG_SmearingJSON']
[0.01588 0.01588 0.0237  ... 0.01588 0.01588 0.01588]


In [93]:
file = up.recreate('GJet_PT-15to6000_TuneCP5-Flat_13p6TeV_20to29_bkg1.root')
file["treeB"] = {"photon_sieip": tot_bkg1.photon_sieip[:,0],
                 "photon_sieie": tot_bkg1.photon_sieie[:,0],
                 "photon_r9": tot_bkg1.photon_r9[:,0],
                 "photon_etaWidth": tot_bkg1.photon_etaWidth[:,0],
                 "photon_phiWidth": tot_bkg1.photon_phiWidth[:,0],
                 "photon_esEnergyOverRawE": tot_bkg1.photon_esEnergyOverRawE[:,0],
                 "photon_esEffSigmaRR": tot_bkg1.photon_esEffSigmaRR[:,0],
                 "photon_pfPhoIso03": tot_bkg1.photon_pfPhoIso03[:,0],
                 "photon_hoe": tot_bkg1.photon_hoe[:,0],
                 "photon_ecalPFClusterIso": tot_bkg1.photon_ecalPFClusterIso[:,0],
                 "photon_hcalPFClusterIso": tot_bkg1.photon_hcalPFClusterIso[:,0],
                 "photon_energyRaw":   tot_bkg1.photon_energyRaw[:,0],
                 "photon_pfChargedIsoPFPV":tot_bkg1.photon_pfChargedIsoPFPV[:,0],
                 "photon_pfChargedIsoWorstVtx": tot_bkg1.photon_pfChargedIsoWorstVtx[:,0],
                 "photon_s4": tot_bkg1.photon_s4[:,0],
                 "photon_sipip": tot_bkg1.photon_sipip[:,0],
                 "photon_rho": rho1,
                 "photon_pt": tot_bkg1.photon_pt[:,0],
                 "photon_eta": tot_bkg1.photon_eta[:,0],
                 "photon_isScEtaEB": tot_bkg1.photon_isScEtaEB[:,0],
                 "photon_isScEtaEE": tot_bkg1.photon_isScEtaEE[:,0],
                 "isPhoton": tot_bkg1.isPhoton[:,0]
                }
pho_pt = tot_bkg1.photon_pt[:,0]
pho_eta = tot_bkg1.photon_eta[:,0]
pho_isScEtaEB = tot_bkg1.photon_isScEtaEB[:,0]
pho_isScEtaEE = tot_bkg1.photon_isScEtaEE[:,0]

h_photon_pt_eb1 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_eb").Weight()
h_photon_eta_eb1 = hist.Hist.new.Reg(10, 0, 1.44, name="photon_eta_eb").Weight()
h_photon_pt_ee1 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_ee").Weight()
h_photon_eta_ee1 = hist.Hist.new.Reg(10, 1.566, 2.5, name="photon_eta_ee").Weight()
h_pt_eta_eb1 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 0, 1.44))
h_pt_eta_ee1 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 1.566, 2.5))
h_pt_eta1 = hist.Hist(hist.axis.Regular(60, 15, 615),hist.axis.Regular(20, 0, 2.5))

h_photon_pt_eb1.fill(pho_pt[pho_isScEtaEB==1])
h_photon_pt_ee1.fill(pho_pt[pho_isScEtaEE==1])
h_photon_eta_eb1.fill(abs(pho_eta[pho_isScEtaEB==1]))
h_photon_eta_ee1.fill(abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta_eb1.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta_ee1.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta1.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta1.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))

file["Photon_pt_EB"] = h_photon_pt_eb1
file["Photon_pt_EE"] = h_photon_pt_ee1
file["Photon_eta_EB"] = h_photon_eta_eb1
file["Photon_eta_EE"] = h_photon_eta_ee1
file["Photon_pt_eta_EB"] = h_pt_eta_eb1
file["Photon_pt_eta_EE"] = h_pt_eta_ee1
file["Photon_pt_eta"] = h_pt_eta1

In [94]:
file = up.recreate('GJet_PT-15to6000_TuneCP5-Flat_13p6TeV_20to29_bkg2.root')
file["treeB"] = {"photon_sieip": tot_bkg2.photon_sieip[:,1],
                 "photon_sieie": tot_bkg2.photon_sieie[:,1],
                 "photon_r9": tot_bkg2.photon_r9[:,1],
                 "photon_etaWidth": tot_bkg2.photon_etaWidth[:,1],
                 "photon_phiWidth": tot_bkg2.photon_phiWidth[:,1],
                 "photon_esEnergyOverRawE": tot_bkg2.photon_esEnergyOverRawE[:,1],
                 "photon_esEffSigmaRR": tot_bkg2.photon_esEffSigmaRR[:,1],
                 "photon_pfPhoIso03": tot_bkg2.photon_pfPhoIso03[:,1],
                 "photon_hoe": tot_bkg2.photon_hoe[:,1],
                 "photon_ecalPFClusterIso": tot_bkg2.photon_ecalPFClusterIso[:,1],
                 "photon_hcalPFClusterIso": tot_bkg2.photon_hcalPFClusterIso[:,1],
                 "photon_energyRaw":   tot_bkg2.photon_energyRaw[:,1],
                 "photon_pfChargedIsoPFPV":tot_bkg2.photon_pfChargedIsoPFPV[:,1],
                 "photon_pfChargedIsoWorstVtx": tot_bkg2.photon_pfChargedIsoWorstVtx[:,1],
                 "photon_s4": tot_bkg2.photon_s4[:,1],
                 "photon_sipip": tot_bkg2.photon_sipip[:,1],
                 "photon_rho": rho2,
                 "photon_pt": tot_bkg2.photon_pt[:,1],
                 "photon_eta": tot_bkg2.photon_eta[:,1],
                 "photon_isScEtaEB": tot_bkg2.photon_isScEtaEB[:,1],
                 "photon_isScEtaEE": tot_bkg2.photon_isScEtaEE[:,1],
                 "isPhoton": tot_bkg2.isPhoton[:,1]
                }
pho_pt = tot_bkg2.photon_pt[:,0]
pho_eta = tot_bkg2.photon_eta[:,0]
pho_isScEtaEB = tot_bkg2.photon_isScEtaEB[:,0]
pho_isScEtaEE = tot_bkg2.photon_isScEtaEE[:,0]

h_photon_pt_eb2 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_eb").Weight()
h_photon_eta_eb2 = hist.Hist.new.Reg(10, 0, 1.44, name="photon_eta_eb").Weight()
h_photon_pt_ee2 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_ee").Weight()
h_photon_eta_ee2 = hist.Hist.new.Reg(10, 1.566, 2.5, name="photon_eta_ee").Weight()
h_pt_eta_eb2 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 0, 1.44))
h_pt_eta_ee2 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 1.566, 2.5))
h_pt_eta2 = hist.Hist(hist.axis.Regular(60, 15, 615),hist.axis.Regular(20, 0, 2.5))

h_photon_pt_eb2.fill(pho_pt[pho_isScEtaEB==1])
h_photon_pt_ee2.fill(pho_pt[pho_isScEtaEE==1])
h_photon_eta_eb2.fill(abs(pho_eta[pho_isScEtaEB==1]))
h_photon_eta_ee2.fill(abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta_eb2.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta_ee2.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta2.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta2.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))

file["Photon_pt_EB"] = h_photon_pt_eb2
file["Photon_pt_EE"] = h_photon_pt_ee2
file["Photon_eta_EB"] = h_photon_eta_eb2
file["Photon_eta_EE"] = h_photon_eta_ee2
file["Photon_pt_eta_EB"] = h_pt_eta_eb2
file["Photon_pt_eta_EE"] = h_pt_eta_ee2
file["Photon_pt_eta"] = h_pt_eta2

In [95]:
file = up.recreate('GJet_PT-15to6000_TuneCP5-Flat_13p6TeV_20to29_bkg3.root')
file["treeB"] = {"photon_sieip": tot_bkg3.photon_sieip[:,2],
                 "photon_sieie": tot_bkg3.photon_sieie[:,2],
                 "photon_r9": tot_bkg3.photon_r9[:,2],
                 "photon_etaWidth": tot_bkg3.photon_etaWidth[:,2],
                 "photon_phiWidth": tot_bkg3.photon_phiWidth[:,2],
                 "photon_esEnergyOverRawE": tot_bkg3.photon_esEnergyOverRawE[:,2],
                 "photon_esEffSigmaRR": tot_bkg3.photon_esEffSigmaRR[:,2],
                 "photon_pfPhoIso03": tot_bkg3.photon_pfPhoIso03[:,2],
                 "photon_hoe": tot_bkg3.photon_hoe[:,2],
                 "photon_ecalPFClusterIso": tot_bkg3.photon_ecalPFClusterIso[:,2],
                 "photon_hcalPFClusterIso": tot_bkg3.photon_hcalPFClusterIso[:,2],
                 "photon_energyRaw":   tot_bkg3.photon_energyRaw[:,2],
                 "photon_pfChargedIsoPFPV":tot_bkg3.photon_pfChargedIsoPFPV[:,2],
                 "photon_pfChargedIsoWorstVtx": tot_bkg3.photon_pfChargedIsoWorstVtx[:,2],
                 "photon_s4": tot_bkg3.photon_s4[:,2],
                 "photon_sipip": tot_bkg3.photon_sipip[:,2],
                 "photon_rho": rho3,
                 "photon_pt": tot_bkg3.photon_pt[:,1],
                 "photon_eta": tot_bkg3.photon_eta[:,1],
                 "photon_isScEtaEB": tot_bkg3.photon_isScEtaEB[:,1],
                 "photon_isScEtaEE": tot_bkg3.photon_isScEtaEE[:,1],
                 "isPhoton": tot_bkg3.isPhoton[:,2]
                }
pho_pt = tot_bkg3.photon_pt[:,0]
pho_eta = tot_bkg3.photon_eta[:,0]
pho_isScEtaEB = tot_bkg3.photon_isScEtaEB[:,0]
pho_isScEtaEE = tot_bkg3.photon_isScEtaEE[:,0]

h_photon_pt_eb3 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_eb").Weight()
h_photon_eta_eb3 = hist.Hist.new.Reg(10, 0, 1.44, name="photon_eta_eb").Weight()
h_photon_pt_ee3 = hist.Hist.new.Reg(100, 15, 1015, name="photon_pt_ee").Weight()
h_photon_eta_ee3 = hist.Hist.new.Reg(10, 1.566, 2.5, name="photon_eta_ee").Weight()
h_pt_eta_eb3 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 0, 1.44))
h_pt_eta_ee3 = hist.Hist(hist.axis.Regular(100, 15, 1015),hist.axis.Regular(10, 1.566, 2.5))
h_pt_eta3 = hist.Hist(hist.axis.Regular(60, 15, 615),hist.axis.Regular(20, 0, 2.5))

h_photon_pt_eb3.fill(pho_pt[pho_isScEtaEB==1])
h_photon_pt_ee3.fill(pho_pt[pho_isScEtaEE==1])
h_photon_eta_eb3.fill(abs(pho_eta[pho_isScEtaEB==1]))
h_photon_eta_ee3.fill(abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta_eb3.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta_ee3.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))
h_pt_eta3.fill(pho_pt[pho_isScEtaEB==1],abs(pho_eta[pho_isScEtaEB==1]))
h_pt_eta3.fill(pho_pt[pho_isScEtaEE==1],abs(pho_eta[pho_isScEtaEE==1]))

file["Photon_pt_EB"] = h_photon_pt_eb3
file["Photon_pt_EE"] = h_photon_pt_ee3
file["Photon_eta_EB"] = h_photon_eta_eb3
file["Photon_eta_EE"] = h_photon_eta_ee3
file["Photon_pt_eta_EB"] = h_pt_eta_eb3
file["Photon_pt_eta_EE"] = h_pt_eta_ee3
file["Photon_pt_eta"] = h_pt_eta3